<a href="https://colab.research.google.com/github/adidror005/youtube-videos/blob/main/PolygonIO_Getting_Bar_Data_Actual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Polygon.IO
* Polygon.IO is a super duper data playform https://polygon.io/

* Go to Pricing and subscribe to Free Account
    - 4 types of subscriptions for each Options, Stocks, Indices, Forex
    - Free tier is good enough for a lot. 5 API calls max per minute
* For docs go to docs / client library Python. See https://polygon.io/docs/stocks/getting-started


#### Install Client Library


In [4]:
!pip install polygon-api-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


### API Key
* Get in dashboard https://polygon.io/dashboard
* Copy and paste here to MY_API_KEY. Note I did the "proper" thing by setting a secret, but this isn't necessary and you can just set MY_API_KEY to your API Key string

In [2]:
from google.colab import userdata
MY_API_KEY = userdata.get('POLYGON_API_KEY')

#### Connect to client

In [5]:
from polygon import RESTClient
client = RESTClient(api_key=MY_API_KEY)

### Use client.list_aggs to get timebars
* Call help to see good documentation
* Can get 3 yrs max for historical data for free version
* Since free version has 5 API calls per minute we can sleep every 5 calls for 1 minute.
* Since using Free API has limit to API calls use limit = MAX_LIMIT rows... By default limit here is 5,000, but max is 50,000
* Each call has LIMIT_MAX=50,000 bars so we pause every LIMIT_MAX*5 = 250,000 bars
* "Pagenated" endpoint iterator


In [6]:
help(client.list_aggs)

Help on method list_aggs in module polygon.rest.aggs:

list_aggs(ticker: str, multiplier: int, timespan: str, from_: Union[str, int, datetime.datetime, datetime.date], to: Union[str, int, datetime.datetime, datetime.date], adjusted: Optional[bool] = None, sort: Union[str, polygon.rest.models.common.Sort, NoneType] = None, limit: Optional[int] = None, params: Optional[Dict[str, Any]] = None, raw: bool = False, options: Optional[polygon.rest.models.request.RequestOptionBuilder] = None) -> Union[Iterator[polygon.rest.models.aggs.Agg], urllib3.response.HTTPResponse] method of polygon.rest.RESTClient instance
    List aggregate bars for a ticker over a given date range in custom time window sizes.
    
    :param ticker: The ticker symbol.
    :param multiplier: The size of the timespan multiplier.
    :param timespan: The size of the time window.
    :param from_: The start of the aggregate time window as YYYY-MM-DD, a date, Unix MS Timestamp, or a datetime.
    :param to: The end of the a

In [12]:
import time
MAX_LIMIT = 50_000
bars = []
for bar in client.list_aggs(
    ticker='TSLA',
    multiplier=1,
    timespan='minute',
    from_="2021-01-01",
    to="2024-06-18",
    limit=MAX_LIMIT
):
  bars.append(bar)
  if len(bars)==(MAX_LIMIT*5):
    print(f'sleeping for 1 minute')
    print(f"We have {len(bars)} bars so far")
    time.sleep(60)


sleeping for 1 minute
We have 250000 bars so far


In [13]:
len(bars)

449157

In [14]:
bars[-1]

Agg(open=184.67, high=184.68, low=184.59, close=184.68, volume=2698, vwap=184.6228, timestamp=1718755140000, transactions=60, otc=None)

### Make a dataframe

In [15]:
import pandas as pd
df = pd.DataFrame(bars)
df

,open,high,low,close,volume,vwap,timestamp,transactions,otc
0,222.3333,222.8800,219.6833,222.8000,23694.0,222.2380,1655798400000,494,None
1,222.7100,223.6667,222.7100,223.6667,29304.0,223.2951,1655798460000,503,None
2,223.6667,223.7367,223.2000,223.3333,24825.0,223.4986,1655798520000,418,None
3,223.4000,223.4467,223.2333,223.4467,17655.0,223.4136,1655798580000,321,None
4,223.4000,223.4000,223.3667,223.4000,8940.0,223.4261,1655798640000,186,None
...,...,...,...,...,...,...,...,...,...
449152,184.6700,184.6800,184.5939,184.5939,9647.0,184.6392,1718754900000,59,None
449153,184.6400,184.6400,184.6400,184.6400,490.0,184.6367,1718754960000,28,None
449154,184.6400,184.6800,184.6184,184.6800,1585.0,184.6566,1718755020000,30,None
449155,184.6200,184.6200,184.6000,184.6000,1817.0,184.6110,1718755080000,35,None


### Turn timestamp col into datetime

In [16]:
df['timestamp']=pd.to_datetime(df['timestamp'],unit='ms')
df

,open,high,low,close,volume,vwap,timestamp,transactions,otc
0,222.3333,222.8800,219.6833,222.8000,23694.0,222.2380,2022-06-21 08:00:00,494,None
1,222.7100,223.6667,222.7100,223.6667,29304.0,223.2951,2022-06-21 08:01:00,503,None
2,223.6667,223.7367,223.2000,223.3333,24825.0,223.4986,2022-06-21 08:02:00,418,None
3,223.4000,223.4467,223.2333,223.4467,17655.0,223.4136,2022-06-21 08:03:00,321,None
4,223.4000,223.4000,223.3667,223.4000,8940.0,223.4261,2022-06-21 08:04:00,186,None
...,...,...,...,...,...,...,...,...,...
449152,184.6700,184.6800,184.5939,184.5939,9647.0,184.6392,2024-06-18 23:55:00,59,None
449153,184.6400,184.6400,184.6400,184.6400,490.0,184.6367,2024-06-18 23:56:00,28,None
449154,184.6400,184.6800,184.6184,184.6800,1585.0,184.6566,2024-06-18 23:57:00,30,None
449155,184.6200,184.6200,184.6000,184.6000,1817.0,184.6110,2024-06-18 23:58:00,35,None


##### Note
* If you don't need to worry about API limits you can just put it in df directly, but since we are using Free Tier we will get a max retry error

In [18]:
df_not_full=pd.DataFrame(client.list_aggs(
    ticker='TSLA',
    multiplier=1,
    timespan='minute',
    from_="2021-01-30",
    to="2024-06-18"
))

MaxRetryError: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v2/aggs/ticker/TSLA/range/1/minute/1659731460000/2024-06-18?cursor=bGltaXQ9NTAwMCZzb3J0PWFzYw (Caused by ResponseError('too many 429 error responses'))

### See Adam GetBags Channel For Additional Videos

* https://www.youtube.com/@damgetbags